# Lab 3

Name:

Date: 

**As you work through the code below, respond to all in-line comments and questions.**

Define matrices W1, W2, B1 and B2.  Take the values from In-Class Activity 1.

<img src="https://github.com/eickholtj/CPS580/raw/master/img/weight-matrices.png">

[Matrices can be created](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array) from lists or empty.



In [ ]:
import numpy as np
# W1 = 

#print(W1.shape)
#print(W2.shape)
#print(B1.shape)
#print(B2.shape)

Now let's define X to contain the examples (.1, .8) and (.2, .5).  Then run them through the network defined by W and B.  Use the sigmoid activation energy.  The rule to calculate A is X.W + B and Z is sigmoid(A).

In [ ]:
import math

def sigmoid(x):
    "Numerically stable sigmoid function."
    if x >= 0:
        z = math.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = math.exp(x)
        return z / (1 + z)

X = np.array([[.1,.8], [.2, .5]])

# Let's calculate a forward pass.  Compare the results with what you obtained when 
#  worked these out by hand.
#A1 = 

print("The values of the hidden layer are ")
#print(Z1)

print("The raw output of the network is ")
#print(Z2)


Using a decision threshold of 0.5, generate predictions and calculate the accuracy of the predictions if the true targets are 0 and 1.  

In [ ]:
raw_preds = Z2

# Targets 
targets = np.array([[0], [1]], dtype=np.int32)

# Set the predictions


# Calculate the number of correct predictions
# correct = 
# total = 

print("The number of correct predictions was {} of {} total predictions.".format(correct, total))
print("The accuracy is {}".format(correct / total))

Let's first read in some data using the [loadtxt](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) function of numpy.  This allows you to read in data from a text file (e.g., csv) and will return a ndarray.  The [shape attribute](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.shape.html) provides a tuple with the length in each dimension.    

In [ ]:
import urllib.request as urllib
import numpy as np

# Read in the data from a URL; works for small datasets
url = "https://github.com/eickholtj/CPS580/raw/master/data/game-data.txt"
file = urllib.urlopen(url)
data = np.loadtxt(file)

# Display the type of 'data'.  What are its dimensions?  What type of tensor is
#   it?  



Let's select some some subsets of data using sliding and other [indexing features](https://numpy.org/doc/stable/reference/arrays.indexing.html).

In [ ]:
# Pull out the first 10 rows, the last column, and every other row



                    # start_idx:stop_idx:step_size and empty values are 
                    #   interpreted to be as large as possible


# Split the data into two sets



Now let's randomize the order of the examples (i.e., the rows) in the data.

In [ ]:

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices,:]


Pulling all of these pieces together, we will train a simple neural network with our simulated data and back propagation.  Apply the SimpleNN class to our game data.

In [ ]:
import numpy as np
import sys
import math

def sigmoid(x):
    "Numerically stable sigmoid function."
    if x >= 0:
        z = math.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = math.exp(x)
        return z / (1 + z)

def sigmoid_prime(x):
    return sigmoid(x) * sigmoid(1-x)


class SimpleNN:
    W1 = None
    B1 = None
    W2 = None
    B2 = None
    training_X = None
    training_Y = None

    def __init__(self, MidLayerSize, training_X, training_Y):
        self.W1 = np.random.rand(training_X.shape[1], int(MidLayerSize))
        self.W1 = 0.01 * self.W1 
        self.B1 = np.random.rand(1, int(MidLayerSize))
        self.B1 = 0.01 * self.B1

        self.W2 = np.random.rand(int(MidLayerSize), 1)
        self.W2 = 0.01 * self.W2 
        self.B2 = np.random.rand(1,1)
        self.B2 = 0.01 * self.B2

        self.training_X = training_X
        self.training_Y = training_Y

    def apply(self, X):

        A1 = np.dot(X, self.W1) + self.B1
        Z1 = np.vectorize(sigmoid)(A1)

        A2 = np.dot(Z1, self.W2) + self.B2
        Z2 = np.vectorize(sigmoid)(A2)

        return Z2

    def evaluate(self, X, Y):
        O = self.apply(X)
        
        P = np.zeros(O.shape)
        for i in range(P.shape[0]):
            if O[i] > 0.5:
                P[i] = 1
 
        correct_count = 0
        for i in range(P.shape[0]):
            if abs(Y[i] - P[i]) < 0.001:
               correct_count = correct_count + 1

        print("Accuracy ", 1.0 * correct_count / Y.shape[0])
          
    def train(self, epochs, batch_size, learning_rate):
       for i in range(0, epochs):
           print("Starting epoch ", i + 1)

           for j in range(0, int(self.training_X.shape[0] / batch_size)):
               # print("... batch ", j + 1)

               Xt = self.training_X[j*batch_size:(j+1)*batch_size,:]
               Yt = self.training_Y[j*batch_size:(j+1)*batch_size]

               # Forward pass
               A1 = np.dot(Xt, self.W1) + np.dot(np.ones((Xt.shape[0],1)), self.B1)
               Z1 = np.vectorize(sigmoid)(A1)

               A2 = np.dot(Z1, self.W2) + np.dot(np.ones((Z1.shape[0],1)), self.B2)
               Z2 = np.vectorize(sigmoid)(A2)

               # Calculate D
               D2 = (Z2 - np.reshape(Yt, (Z2.shape[0],1))) * np.vectorize(sigmoid_prime)(A2)
               D1 = np.dot(D2, np.transpose(self.W2))  * np.vectorize(sigmoid_prime)(A1)

               DE_dw2 = np.dot(np.transpose(Z1) , D2)               
               DE_dw1 = np.dot(np.transpose(Xt) , D1)

               # Update weights               
               self.W2 = self.W2 - learning_rate * DE_dw2 
               self.W1 = self.W1 - learning_rate * DE_dw1
               #self.B2 = self.B2 - learning_rate * D2
               #self.B1 = self.B1 - learning_rate * D1
               
           self.evaluate(Xt, Yt)



# Randomize the data, the pull of 80% for training and split into X, Y
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices,:]
N = int(.8 * data.shape[0])

train_X = data[:N,0:2]
train_Y = data[:N,2]
test_X = data[N:,0:2]
test_Y = data[N:,2]

nn = SimpleNN(2, train_X, train_Y)

nn.train(15, 50, 0.01)
nn.apply(test_X)

nn.evaluate(test_X, test_Y)	


Now let's look at some image data.  Keras provides easy access to the [MNIST dataset](https://keras.io/api/datasets/mnist/#load_data-function).

In [ ]:
from tensorflow.keras.datasets import mnist

(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

# How the training image data is packaged?  How are the labels encoded?


# Come up with a way to visualize the data
np.set_printoptions(linewidth=200)



Let's try to develop a classifer that will work on the MNIST data.  We've only seen binary classifiers so far so we need to just pull out two types of digits.

In [ ]:
one_indices = np.where(train_Y[:] == 0)
zero_indices = np.where(train_Y[:] == 1)

one_zero_indices = np.concatenate((one_indices, zero_indices), axis=1).flatten()
np.random.shuffle(one_zero_indices)

subtrain_X = train_X[one_zero_indices,:]
subtrain_Y = train_Y[one_zero_indices]

# Try to figure out how many of each digit there are...

# Reshape X to be a 2d tensor so that it will fit into SimpleNN
subtrain_X = np.reshape(subtrain_X, (subtrain_X.shape[0], -1))

# Normalize the feature in x to be between 0 and 1
subtrain_X = subtrain_X / 255.

nn = SimpleNN(25, subtrain_X, subtrain_Y)
nn.train(15, 50, 0.0001)

# Try to evaluate on data from the test data

**Submission:** Submit this notebook and a PDF of its generated output via Blackboard by the end of the day.  Submit the files individually.


